## 九. RAG-增强检索生成 (Retrieval-Augmented Generation)

前面提到的 引入外部知识库的文档问答（QA based Documents）已经算是一个最基础的 RAG 了，这里再扩展一下，增加两个重要特性：
1. 在context 中 引入来自互联网搜索的知识
2. 使用自定义LLM

1. web search:

In [ ]:
# 联网搜索引擎使用 duckduckgo
!pip install duckduckgo_search

In [1]:
# 先来简单使用下 duckduckgo
from duckduckgo_search import DDGS

# test
with DDGS() as ddgs:
    # for r in ddgs.text('床前明月光', region='cn-zh', safesearch='off', timelimit='y', max_results=5):
    #     print(r)
    for r in ddgs.text('who is the author of alice in wonderland?', region='us-en', safesearch='off', timelimit='y', max_results=3):
        print(r)

{'title': 'Lewis Carroll | Who Was The Author Of Alice In Wonderland ...', 'href': 'https://www.historyextra.com/period/victorian/lewis-carroll-author-life-alice-in-wonderland/', 'body': "On 14 January it's the 125th anniversary of the death of Lewis Carroll, mathematician, polymath, and author of Alice's Adventures in Wonderland. His book has captured our imaginations for over 150 years, and has been enjoyed around the world. But who was the man behind the beloved tale?"}
{'title': 'Alice in Wonderland | Summary, Author & Adaptations | Study.com', 'href': 'https://study.com/academy/lesson/alice-in-wonderland-lewis-carroll-author-summary-analysis.html', 'body': 'Quiz Course What is Alice in Wonderland About? Young Alice sits by a riverbank and sees a white rabbit curiously wearing a coat and holding a pocket watch. The White Rabbit cryptically states...'}
{'title': 'The real story and places behind Alice in Wonderland', 'href': 'https://britishheritage.com/travel/real-story-alice-wonde

In [2]:
def search_web(keywords, region='us-en', max_results=3):
    web_content = ''
    with DDGS() as ddgs:
        ddgs_gen = ddgs.text(keywords=keywords, region=region, safesearch='off', timelimit='y', backend="api", max_results=max_results)
        for r in ddgs_gen:
            web_content += (r['body'] + "\n")

    return web_content

2. custom llm:
- 使用minimax api

In [3]:
import os
from typing import Dict, List, Optional, Tuple, Union

from langchain.llms.base import LLM

import requests

class MinimaxLLM(LLM):
    max_token: int = 1024
    model: str = "abab5.5-chat"
    group_id: str = ""
    api_key: str = ""
    history = []

    def __init__(self, group_id, api_key):
        
        super().__init__()
        self.group_id = group_id
        self.api_key = api_key

    @property
    def _llm_type(self) -> str:
        return "MinimaxApi"

    def _call(self,
              prompt: str,
              stop: Optional[List[str]] = None) -> str:
        

        # group_id = os.getenv('group_id')
        # api_key = os.getenv('api_key')

        url = f'https://api.minimax.chat/v1/text/chatcompletion?GroupId={self.group_id}'
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        request_body = {
            "model": self.model,
            "tokens_to_generate": self.max_token,
            'messages': [],
            # "temperature": 0.9,
            # "top_p": 0.95,
        }

    
        for i in self.history:
            h_input = i[0]
            h_reply = i[1]
            request_body['messages'].append({
                "sender_type": "USER",
                "text": h_input
            })
            request_body['messages'].append({"sender_type": "BOT", "text": h_reply})

        request_body['messages'].append({"sender_type": "USER", "text": prompt})
        
        # from pprint import pprint
        # pprint( request_body)
        resp = requests.post(url, headers=headers, json=request_body)
        

        response = resp.json()['reply']
        #  将当次的ai回复内容加入messages
        request_body['messages'].append({"sender_type": "BOT", "text": response})
        self.history.append((prompt, response))
        
      
        return response

In [4]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [5]:
import os

# TEST
llm = MinimaxLLM(os.getenv("MINIMAX_GROUP_ID"), os.getenv("MINIMAX_API_KEY"))
s = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nBut her sister sat still just as she left her, leaning her head on her hand, watching the setting sun, and thinking of little Alice and all her wonderful Adventures, till she too began dreaming after a fashion, and this was her dream:— First, she dreamed of little Alice herself, and once again the tiny hands were clasped upon her knee, and the bright eager eyes were looking up into hers—she could hear the very tones of her voice, and see that queer little toss of her head to keep back the wandering hair that would always get into her eyes—and still as she listened, or seemed to listen, the whole place around her became alive with the strange creatures of her little sister’s dream. The long grass rustled at her feet as the White Rabbit hurried by—the frightened Mouse splashed his way through the neighbouring pool—she could hear the rattle of the teacups as the March Hare and his friends shared their never-ending meal, and the shrill voice of the Queen ordering off her unfortunate guests to execution—once more the pig-baby was sneezing on the Duchess’s knee, while plates and dishes crashed around it—once more the shriek of the Gryphon, the squeaking of the Lizard’s slate-pencil, and the choking of the suppressed guinea-pigs, filled the air, mixed up with the distant sobs of the miserable Mock Turtle. \nAlice could see, as well as if she were looking over their shoulders, that all the jurors were writing down “stupid things!” on their slates, and she could even make out that one of them didn’t know how to spell “stupid,” and that he had to ask his neighbour to tell him. “A nice muddle their slates’ll be in before the trial’s over!” thought Alice. One of the jurors had a pencil that squeaked. This of course, Alice could not stand, and she went round the court and got behind him, and very soon found an opportunity of taking it away. She did it so quickly that the poor little juror (it was Bill, the Lizard) could not make out at all what had become of it; so, after hunting all about for it, he was obliged to write with one finger for the rest of the day; and this was of very little use, as it left no mark on the slate. “Herald, read the accusation!” said the King. On this the White Rabbit blew three blasts on the trumpet, and then unrolled the parchment scroll, and read as follows:\n\nIt was the White Rabbit, trotting slowly back again, and looking anxiously about as it went, as if it had lost something; and she heard it muttering to itself “The Duchess! The Duchess! Oh my dear paws! Oh my fur and whiskers! She’ll get me executed, as sure as ferrets are ferrets! Where can I have dropped them, I wonder?” Alice guessed in a moment that it was looking for the fan and the pair of white kid gloves, and she very good-naturedly began hunting about for them, but they were nowhere to be seen—everything seemed to have changed since her swim in the pool, and the great hall, with the glass table and the little door, had vanished completely. Very soon the Rabbit noticed Alice, as she went hunting about, and called out to her in an angry tone, “Why, Mary Ann, what are you doing out here? Run home this moment, and fetch me a pair of gloves and a fan! Quick, now!” And Alice was so much frightened that she ran off at once in the direction it pointed to, without trying to explain the mistake it had made. “He took me for his housemaid,” she said to herself as she ran. “How surprised he’ll be when he finds out who I am! But I’d better take him his fan and gloves—that is, if I can find them.” As she said this, she came upon a neat little house, on the door of which was a bright brass plate with the name “W. RABBIT,” engraved upon it. She went in without knocking, and hurried upstairs, in great fear lest she should meet the real Mary Ann, and be turned out of the house before she had found the fan and gloves.\n\nAlice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?” So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge.\nAfter a time she heard a little pattering of feet in the distance, and she hastily dried her eyes to see what was coming. It was the White Rabbit returning, splendidly dressed, with a pair of white kid gloves in one hand and a large fan in the other: he came trotting along in a great hurry, muttering to himself as he came, “Oh! the Duchess, the Duchess! Oh! won’t she be savage if I’ve kept her waiting!” Alice felt so desperate that she was ready to ask help of any one; so, when the Rabbit came near her, she began, in a low, timid voice, “If you please, sir—” The Rabbit started violently, dropped the white kid gloves and the fan, and skurried away into the darkness as hard as he could go. Alice took up the fan and gloves, and, as the hall was very hot, she kept fanning herself all the time she went on talking: “Dear, dear! How queer everything is to-day! And yesterday things went on just as usual. I wonder if I’ve been changed in the night? Let me think: was I the same when I got up this morning? I almost think I can remember feeling a little different. But if I’m not the same, the next question is, Who in the world am I? Ah, that’s the great puzzle!” And she began thinking over all the children she knew that were of the same age as herself, to see if she could have been changed for any of them.\n\nThey had not gone far before they saw the Mock Turtle in the distance, sitting sad and lonely on a little ledge of rock, and, as they came nearer, Alice could hear him sighing as if his heart would break. She pitied him deeply. “What is his sorrow?” she asked the Gryphon, and the Gryphon answered, very nearly in the same words as before, “It’s all his fancy, that: he hasn’t got no sorrow, you know. Come on!” So they went up to the Mock Turtle, who looked at them with large eyes full of tears, but said nothing. “This here young lady,” said the Gryphon, “she wants for to know your history, she do.” “I’ll tell it her,” said the Mock Turtle in a deep, hollow tone: “sit down, both of you, and don’t speak a word till I’ve finished.” So they sat down, and nobody spoke for some minutes. Alice thought to herself, “I don’t see how he can ever finish, if he doesn’t begin.” But she waited patiently. “Once,” said the Mock Turtle at last, with a deep sigh, “I was a real Turtle.” These words were followed by a very long silence, broken only by an occasional exclamation of “Hjckrrh!” from the Gryphon, and the constant heavy sobbing of the Mock Turtle. Alice was very nearly getting up and saying, “Thank you, sir, for your interesting story,” but she could not help thinking there must be more to come, so she sat still and said nothing.\nWhen she got back to the Cheshire Cat, she was surprised to find quite a large crowd collected round it: there was a dispute going on between the executioner, the King, and the Queen, who were all talking at once, while all the rest were quite silent, and looked very uncomfortable. The moment Alice appeared, she was appealed to by all three to settle the question, and they repeated their arguments to her, though, as they all spoke at once, she found it very hard indeed to make out exactly what they said. The executioner’s argument was, that you couldn’t cut off a head unless there was a body to cut it off from: that he had never had to do such a thing before, and he wasn’t going to begin at his time of life. The King’s argument was, that anything that had a head could be beheaded, and that you weren’t to talk nonsense. The Queen’s argument was, that if something wasn’t done about it in less than no time she’d have everybody executed, all round. (It was this last remark that had made the whole party look so grave and anxious.) Alice could think of nothing else to say but “It belongs to the Duchess: you’d better ask her about it.” “She’s in prison,” the Queen said to the executioner: “fetch her here.” And the executioner went off like an arrow. The Cat’s head began fading away the moment he was gone, and, by the time he had come back with the Duchess, it had entirely disappeared; so the King and the executioner ran wildly up and down looking for it, while the rest of the party went back to the game.\n\nQuestion: What does the author describe the Alice following with?\nHelpful Answer:"
llm(s)


'The author describes Alice following the White Rabbit with her eyes.'

知识库加载与嵌入：

In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('../data/wonderland.txt') # 载入一个长文本，我们还是使用爱丽丝漫游仙境这篇小说作为输入
doc = loader.load()
print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

# 将小说分割成多个部分
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

# 获取字符的总数，以便可以计算平均值
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")


# Embed 文档，然后使用伪数据库将文档和原始文本结合起来
from langchain.vectorstores import FAISS


from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    openai_api_base=os.getenv("AZURE_OPENAI_BASE_URL"),    
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",
    deployment=os.getenv("AZURE_DEPLOYMENT_NAME_EMBEDDING"),
    )

vector_store = FAISS.from_documents(docs, embeddings)

You have 1 document
You have 13638 characters in that document
Now you have 6 documents that have an average of 2,272 characters (smaller pieces)


chain:

In [7]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


def get_knowledge_based_answer(query,
                               vector_store,
                               VECTOR_SEARCH_TOP_K,
                               history_len,
                               temperature,
                               top_p,
                               chat_history=[]):
    
    web_content = search_web(query)

    prompt_template = f"""基于以下已知信息，简洁和专业的来回答末尾的问题。
                        如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分。
                        已知网络检索内容：{web_content}""" + """
                        已知本地知识库内容:
                        {context}
                        问题:
                        {question}"""   
        
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    #llm.history = chat_history[-history_len:] if history_len > 0 else []

    
    knowledge_chain = RetrievalQA.from_llm(
        llm=llm,
        #retriever=vector_store.as_retriever(search_kwargs={"k": VECTOR_SEARCH_TOP_K}),
        retriever=vector_store.as_retriever(),
        prompt=prompt, 
        verbose=True)    

    knowledge_chain.combine_documents_chain.document_prompt = PromptTemplate(
        input_variables=["page_content"], template="{page_content}")

    knowledge_chain.return_source_documents = True


    print(f"-> web_content: {web_content}, prompt: {prompt}, query: {query}" )
    result = knowledge_chain({"query": query})
    return result

In [8]:
query = "Who is the author of the Alice in Wonderland?"
resp = get_knowledge_based_answer(
    query=query,
    vector_store=vector_store,
    VECTOR_SEARCH_TOP_K=6,
    chat_history=[],
    history_len=0,
    temperature=0.1,
    top_p=0.9,
)
print(resp)

-> web_content: Attempting to capture Christ Church Cathedral's spire with the new technology, he found himself surrounded by the curious children of Henry Liddell, the dean of the college- including Alice Pleasance Liddell (he had already made the acquaintance of Alice's brother Harry, whom he tutored in mathematics).
Alice in Wonderland is the short name for Lewis Carroll 's famous children's book, Alice's Adventures in Wonderland. The novel follows the titular Alice as she initially follows a white...
Deepthi Reddy on 11 February 2021 ; Updated on 20 April 2023 Sub-edited by Isobel Murphy 6 mins to read Lewis Carroll was an English novelist, famous for children's fiction stories. The real name of the author was Charles Lutwidge Dodgson. He wrote using the pen name Lewis Carroll.
, prompt: input_variables=['context', 'question'] template='基于以下已知信息，简洁和专业的来回答末尾的问题。\n                        如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分。\n                        已知网络检索内容：At